In [1]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate 

In [2]:
# This path will be active after the launch of the hackathon
teamname = '9417-brhuang-unsw'
data_folder='s3://tf-trachack-data/212/'
# change root_folder to your team's root folder
# s3://tf-trachack-notebooks/<this should be replaced by team name as provided in EMAIL>/jupyter/jovyan/
root_folder='s3://tf-trachack-notebooks/'+'9417-brhuang-unsw'+'/jupyter/jovyan/'



In [3]:
df = pd.read_csv(root_folder+"sample-notebook/final_dataset.csv")  

In [4]:
print(df.shape)

(55868, 90)


In [5]:
df.drop(['date_observed', 'sim_size'], axis = 1, inplace = True)

In [6]:
df.columns

Index(['line_id', 'upgrade', 'quantity', 'total_quantity', 'lrp_enrolled',
       'length_of_membership', 'last_interaction', 'carrier_carrier 1',
       'carrier_carrier 2', 'carrier_carrier 3', 'plan_name_Other',
       'plan_name_not_given', 'plan_name_plan 1', 'plan_name_plan 2',
       'plan_name_plan 3', 'plan_name_plan 4', 'total_redemptions',
       'total_suspensions', 'total_unsuspensions', 'total_deactivations',
       'total_reactivations', 'reactivation_channel_APP',
       'reactivation_channel_IVR', 'reactivation_channel_SMS',
       'reactivation_channel_TAS', 'reactivation_channel_WEB',
       'reactivation_channel_other', 'cpu_cores_2', 'cpu_cores_2+4',
       'cpu_cores_4', 'cpu_cores_8', 'cpu_cores_none', 'cpu_cores_other',
       'expandable_storage_-1.0', 'expandable_storage_0.0',
       'expandable_storage_1.0', 'gsma_device_type_Smartphone',
       'gsma_device_type_none', 'gsma_device_type_other',
       'gsma_operating_system_Android', 'gsma_operating_system_i

In [7]:
Y = df['upgrade']
X = df.drop(['upgrade', 'line_id'], axis = 1)

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True, random_state = 2)


In [9]:
print(X_train.head())

       quantity  total_quantity  lrp_enrolled  length_of_membership  \
15764       0.0             0.0             0                   0.0   
14260       0.0             0.0             0                   0.0   
14436       0.0             0.0             0                   0.0   
21640       0.0             0.0             0                   0.0   
363         0.0             0.0             1                1122.0   

       last_interaction  carrier_carrier 1  carrier_carrier 2  \
15764               0.0                  0                  0   
14260               0.0                  1                  0   
14436               0.0                  1                  0   
21640               0.0                  0                  0   
363              1122.0                  1                  0   

       carrier_carrier 3  plan_name_Other  plan_name_not_given  ...  \
15764                  1                0                    0  ...   
14260                  0                

In [10]:
print(X_test.head())

       quantity  total_quantity  lrp_enrolled  length_of_membership  \
8526        0.0             0.0             0                   0.0   
18095       0.0             0.0             0                   0.0   
17383       0.0             0.0             0                   0.0   
2186        0.0             0.0             1                1221.0   
43243       0.0             0.0             1                 989.0   

       last_interaction  carrier_carrier 1  carrier_carrier 2  \
8526                0.0                  1                  0   
18095               0.0                  1                  0   
17383               0.0                  1                  0   
2186             1221.0                  1                  0   
43243             989.0                  1                  0   

       carrier_carrier 3  plan_name_Other  plan_name_not_given  ...  \
8526                   0                0                    0  ...   
18095                  0                

In [ ]:
#This gives f1 score of 0.3145290047897818, so I guess more layers doesn't equal better
#mlp = MLPClassifier(hidden_layer_sizes=(50,50,50,50), activation='relu', solver='adam', max_iter=200, random_state = 2)
#This gives f1 score of 0.056038647342995164 (trash)
#mlp = MLPClassifier(hidden_layer_sizes=(8,8,8), activation='relu', solver='adam', max_iter=200, alpha = 0.05, random_state = 2)
#This gives f1 score of 0.49348534201954397
#mlp = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', learning_rate = 'constant', max_iter=500, alpha = 0.05, random_state = 2)

In [11]:
#1st try
#This gives f1 score of 0.2090935418476687 and accuracy of 0.755593341686057
mlp = MLPClassifier(hidden_layer_sizes=(8,8,8), activation='relu', solver='adam', max_iter=500, random_state = 2)
mlp.fit(X_train,Y_train)


MLPClassifier(hidden_layer_sizes=(8, 8, 8), max_iter=500, random_state=2)

In [12]:
predict_train = mlp.predict(X_train)
predict_test = mlp.predict(X_test)

In [13]:
print(confusion_matrix(Y_train,predict_train))
print(classification_report(Y_train,predict_train))

[[32025   469]
 [10781  1419]]
              precision    recall  f1-score   support

           0       0.75      0.99      0.85     32494
           1       0.75      0.12      0.20     12200

    accuracy                           0.75     44694
   macro avg       0.75      0.55      0.53     44694
weighted avg       0.75      0.75      0.67     44694



In [14]:
acc = accuracy_score(Y_test, predict_test)
print(acc)
f1 = f1_score(Y_test, predict_test)
print(f1)

0.755593341686057
0.2090935418476687


In [12]:
#Trying different parameters
mlp = MLPClassifier(max_iter=100, random_state = 2)

In [13]:
#Setting the parameters to grid search over
parameter_space = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,), (25,25), (50,50)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

In [ ]:
clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3, scoring = 'f1')
clf.fit(X_train,Y_train)

In [ ]:
# Best parameter set
print('Best parameters found:\n', clf.best_params_) 
#Best parameters were:
# 'activation': 'relu', 'alpha': 0.05, 'hidden_layer_sizes': (50, 50), 'learning_rate': 'constant', 'solver': 'adam'

In [ ]:
predict_train = clf.predict(X_train)
predict_test = clf.predict(X_test)

acc = accuracy_score(Y_test, predict_test)
print(acc)
f1 = f1_score(Y_test, predict_test)
print(f1)

In [181]:
#Trying different parameters
mlp = MLPClassifier(max_iter=100, random_state = 2)

In [21]:
#THIS TAKES EXTREMELY LONG TO RUN
parameter_space = {
    'hidden_layer_sizes': [(10,10,10), (25,50,25), (50,), (100,100,100), (8,8,8,8), (8,8,8,8,8)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'beta_1': [0.9, 0.95, 0.999],
    'beta_2': [0.9, 0.95, 0.999],
    'learning_rate': ['constant','adaptive'],
}

In [ ]:
#THIS TAKES EXTREMELY LONG TO RUN
clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3, scoring = 'f1')
clf.fit(X_train,Y_train)

In [114]:
predict_train = mlp.predict(X_train)
predict_test = mlp.predict(X_test)

In [ ]:
# Best parameter set
print('Best parameters found:\n', clf.best_params_)

In [ ]:
predict_train = clf.predict(X_train)
predict_test = clf.predict(X_test)

acc = accuracy_score(Y_test, predict_test)
print(acc)
f1 = f1_score(Y_test, predict_test)
print(f1)